**Import Library**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as im
import sympy as sp
from PIL import Image, ImageOps
import os

**Function**

In [47]:
#Process the Image into a flat 1D array
def ProcessImage(img):
    Grascale = ImageOps.grayscale(img)
    Facematrix = im.pil_to_array(Grascale)
    FlatFaceArray = np.concat(Facematrix,axis=None)
    return FlatFaceArray

#Make the Data Matrix

def Datamatrix(clsdir: str, num_files: int):
    for i in range(num_files):
        imgdir = clsdir +"\\" + str(i+1) + ".pgm"
    #Nhap hinh anh
        Face = Image.open(imgdir)
    #Process hinh anh
        FlatFaceArray = ProcessImage(Face)
        if i == 0:
            FaceMatrix = FlatFaceArray
        else:
            FaceMatrix = np.column_stack((FaceMatrix,FlatFaceArray))
    return FaceMatrix

#Make a Data Matrix for a class k
def ClassMatrix(k: int):
    clsdir = "archive\\s" + str(k)
    num_files = sum(1 for entry in os.scandir(clsdir) if entry.is_file())
    return Datamatrix(clsdir, num_files)
ClassMatrix(2).shape

#number of class
num_class = sum(1 for entry in os.scandir("archive") if not entry.is_file())


**Size of the data**

In [42]:
imgdir = "archive\\s1\\1.pgm"
Face= Image.open(imgdir)
FaceArray = im.pil_to_array(Face)
Height, Width = FaceArray.shape


**Make the overall Data Matrix**

In [39]:
#numer of class
num_class = sum(1 for entry in os.scandir("archive") if not entry.is_file())

for i in range(num_class):
    if i == 0:
        FullMatrix = ClassMatrix(i+1)
    else:
        FullMatrix = np.append(FullMatrix, ClassMatrix(i+1),axis=1)
FullMatrix.shape

(10304, 400)

In [ ]:
#The average face
MeanFace=np.mean(FullMatrix, axis=1)
ColumnMeanFace= MeanFace[:, np.newaxis]
print(MeanFace)


[85.6175 85.5775 85.925  ... 76.87   75.865  75.2275]


(10304,)

In [ ]:
CenteredFullMatrix=FullMatrix  - ColumnMeanFace
#Find the Eigenface of the Data
BtB=CenteredFullMatrix.T @ CenteredFullMatrix
V, S, Vt = np.linalg.svd(BtB)
V
#Find the needed amount of eigenvalues
eigsum=np.sum(S)
csum=0
for i in range(S.shape[0]):
    csum +=S[i]
    if csum > 0.9*eigsum:
        e90=i
        break

S_matrix=np.sqrt(np.diag(S))
S_reduced=S_matrix[0:r,:r]
S_inv=np.linalg.inv(S_matrix)
U=CenteredFullMatrix @ V @ S_inv
e90


160000

In [ ]:
#Reconstruct the Eigenface as a picture
Eigenface=U @ S_matrix +ColumnMeanFace
face = Eigenface[:,1]
face = np.split(face, Height)
for i in range(len(face)):
    if i == 0:
        recface=face[0]
    else:
        recface = np.vstack((recface,face[i]))
recim = Image.fromarray(recface).convert("L")
recim.save('test.png','PNG')

In [507]:
#let G be the face that need to be recognize
G = np.random.rand(U[:,1].shape[0])
# the cotribution of a eigenface to it is calculated by
#Let Omega be the contribution vector

def Facespace(vector):
    Omega = np.array([])
    for i in range(num_files):
        diff=U[:,i] @ vector
        Omega = np.append(Omega,[[diff]] )
    return Omega
#Omega= Omega[:, np.newaxis]
#Find the threshhold for the picture to be identified as a face, let that be epsilon
epsilon=0
S = 0
for i in range(num_files):
    S+= Facespace(CenteredFaceMatrix[:,i])
    avgOmega = S/len(range(num_files))
for i in range(num_files):
    e = np.linalg.norm(Facespace(CenteredFaceMatrix[:,i])-avgOmega)
    if e > epsilon:
        epsilon = e

In [508]:

#Input = ProcessImage(Image.open("32.JPG"))-MeanFace
#Input.shape

image=Image.open("32.JPG")
Omega = Facespace(ProcessImage(image)-MeanFace)
epsilon > np.linalg.norm(Omega-avgOmega)


np.False_